In [1]:
!git clone https://github.com/nateraw/huggingface-vit-finetune.git
%cd huggingface-vit-finetune/
!pip install -r requirements.txt
!pip install git+https://github.com/huggingface/transformers.git@master --upgrade
!pip install torchattacks

Cloning into 'huggingface-vit-finetune'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 24 (delta 10), reused 19 (delta 6), pack-reused 0
Unpacking objects: 100% (24/24), done.
/content/huggingface-vit-finetune
     |████████████████████████████████| 849kB 7.1MB/s 
     |████████████████████████████████| 112kB 25.6MB/s 
     |████████████████████████████████| 276kB 15.6MB/s 
     |████████████████████████████████| 276kB 25.4MB/s 
     |████████████████████████████████| 829kB 26.3MB/s 
     |████████████████████████████████| 1.3MB 38.5MB/s 
     |████████████████████████████████| 143kB 52.9MB/s 
     |████████████████████████████████| 296kB 50.7MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=1fdd20252bcd0385e65ecd1b3d2de655b52c4561d139b513135fd1f9b2833468
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from transformers import ViTFeatureExtractor, ViTForImageClassification
import requests

import torchattacks
import time
import json

In [3]:
result_dict = {}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [5]:
transform = transforms.Compose(
    [
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
cifar_trainloader = torch.utils.data.DataLoader(cifar_trainset, batch_size=32, shuffle=True, num_workers=2)
cifar_testloader = torch.utils.data.DataLoader(cifar_testset, batch_size=32, shuffle=False, num_workers=2)
cifar_classes = [
    'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'
]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [26]:
model = ViTForImageClassification.from_pretrained('nateraw/vit-base-patch16-224-cifar10', return_dict=False)
model.to(device)
# model = nn.Sequential(
#     nn.Upsample(scale_factor=7, mode='bilinear'),
#     model
# )


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): PatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0): ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_fea

In [7]:
class Transformer(nn.Module):
  def __init__(self, model):
    super(Transformer, self).__init__()
    self.model = model
  def forward(self, x):
    return self.model(x)[0]


In [27]:
ViT_model = Transformer(model).to(device)

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ViT_model.parameters(), lr=0.0002)

In [28]:
# cifar-10 DATASET consists of 60000 images
correct = 0
total = 0
with torch.no_grad():
  for i, (inputs, labels) in enumerate(cifar_testloader):
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
      outputs = ViT_model(inputs)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    if(i % 10 == 0):
      print('Accuracy of the network on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

print('Accuracy of the network on %d cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))
result_dict["Transformer"] = 100 * correct / total

Accuracy of the network on 32 cifar-10 test images: 100.000000 %
Accuracy of the network on 352 cifar-10 test images: 99.431818 %
Accuracy of the network on 672 cifar-10 test images: 99.553571 %
Accuracy of the network on 992 cifar-10 test images: 99.596774 %


KeyboardInterrupt: ignored

In [17]:
correct = 0
total = 0
with torch.no_grad():
  for i, (inputs, labels) in enumerate(cifar_trainloader):
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
      outputs = ViT_model(inputs)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    if(i % 10 == 0):
      print('Accuracy of the network on %d cifar-10 train images: %f %%' % (total, 100 * correct / total))

print('Accuracy of the network on %d cifar-10 train images: %f %%' % (
    total,
    100 * correct / total))
result_dict["Transformer"] = 100 * correct / total

Accuracy of the network on 32 cifar-10 train images: 100.000000 %
Accuracy of the network on 352 cifar-10 train images: 99.715909 %
Accuracy of the network on 672 cifar-10 train images: 99.851190 %


KeyboardInterrupt: ignored

In [29]:
# Train on clean train data
t0 = time.time()

i = 0
correct = 0
total = 0
running_loss = 0.0
for (inputs, labels) in cifar_trainloader:
  # inputs & labels already on device
  # Adv training
  optimizer.zero_grad()
  inputs = inputs.to(device)
  labels = labels.to(device)

  # forward + backward + optimize
  outputs = ViT_model(inputs)
  loss = criterion(outputs, labels)
  loss.backward()
  optimizer.step()
  running_loss += loss.item()
  
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  if i == 0:
    print(predicted, labels, "\n", correct, total)
  # print("Here", np.sum(predicted == labels))
  if i % 150 == 0:
    print('Training accuracy of ViT on %d clean train data: %f %%' % (total, 100 * correct / total))
    print("running loss: ", running_loss)
  i += 1

print('Training accuracy on %d clean cifar-10 train images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ep1_train_acc"] = 100 * correct / total
t1 = time.time()
result_dict["ViT_ep1_train_sec"] = t1 - t0
result_dict

tensor([0, 8, 9, 5, 1, 5, 4, 0, 3, 9, 0, 2, 1, 2, 3, 3, 1, 9, 3, 0, 4, 8, 3, 2,
        3, 9, 7, 5, 2, 4, 3, 5], device='cuda:0') tensor([0, 8, 9, 5, 1, 5, 4, 0, 3, 9, 0, 2, 1, 2, 3, 3, 1, 9, 3, 0, 4, 8, 3, 2,
        3, 9, 7, 5, 2, 4, 3, 5], device='cuda:0') 
 32 32
Training accuracy of ViT on 32 clean train data: 100.000000 %
running loss:  0.005403601564466953
Training accuracy of ViT on 4832 clean train data: 100.000000 %
running loss:  0.8251057714223862
Training accuracy of ViT on 9632 clean train data: 99.989618 %
running loss:  1.6993686081841588
Training accuracy of ViT on 14432 clean train data: 99.986142 %
running loss:  2.764285245910287


KeyboardInterrupt: ignored

# ATTA Prepare Evaluation Adversarial Examples


In [ ]:
atk_pgd_5 = torchattacks.PGD(ViT_model, eps=8/255, alpha=2/255, steps=5)
atk_pgd_10 = torchattacks.PGD(ViT_model, eps=8/255, alpha=2/255, steps=10)
atk_pgd_20 = torchattacks.PGD(ViT_model, eps=8/255, alpha=2/255, steps=20)

In [ ]:
cifar_adv_test10 = []
start = time.time()
for i, (inputs, labels) in enumerate(cifar_testloader):
    adv_images = atk_pgd_10(inputs, labels)
    cifar_adv_test10.append((adv_images.cpu(), labels.cpu()))
    if i % 100 == 0:
      print("Progress(%): ", i*64/100)

print("total time(sec) : %.2f" % (time.time() - start))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Progress(%):  0.0
total time(sec) : 3522.79


In [ ]:
# Store pgd 10 attacks
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd_10_adv_test.pt'
!touch $outfile_name
torch.save(cifar_adv_test10, outfile_name)
!ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

pgd_10_adv_test.pt   pgd_5_adv_itr1_model.pt  README.gdoc
pgd_5_adv_itr1.json  pgd_5_adv_itr1.pt


In [ ]:
# Load pgd 10 attacks
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd_10_adv_test.pt'
cifar_adv_test10 = torch.load(outfile_name)
len(cifar_adv_test10)

157

In [ ]:
cifar_adv_test20 = []
start = time.time()
for i, (inputs, labels) in enumerate(cifar_testloader):
    adv_images = atk_pgd_20(inputs, labels)
    cifar_adv_test20.append((adv_images.cpu(), labels.cpu()))
    if i % 100 == 0:
      print("Progress(%): ", i*64/100)

print("total time(sec) : %.2f" % (time.time() - start))

In [ ]:
# Store pgd 20 attacks
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd_20_adv_test.pt'
!touch $outfile_name
torch.save(cifar_adv_test20, outfile_name)
!ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

In [ ]:
# Load pgd 20 attacks
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd_20_adv_test.pt'
cifar_adv_test20 = torch.load(outfile_name)
len(cifar_adv_test20)

157

# ATTA Training Epoch 1

In [ ]:
# Generate first itr adv images
cifar_adv = []
start = time.time()
for i, (inputs, labels) in enumerate(cifar_trainloader):
    adv_images = atk_pgd_5(inputs, labels)
    cifar_adv.append((adv_images.cpu(), labels.cpu()))
    if i % 100 == 0:
      print("Progress(%): ", 64*i/500)

print("total time(sec) : %.2f" % (time.time() - start))

# Store
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd_5_adv_itr1_v2.pt'
!touch $outfile_name
torch.save(cifar_adv, outfile_name)
!ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


total time(sec) : 2448.80
pgd_10_adv_test.pt	 pgd_5_adv_itr1_v2.pt	  pgd_5_adv_itr4_model.pt
pgd_20_adv_test.pt	 pgd_5_adv_itr2_model.pt  pgd_5_adv_itr4.pt
pgd_5_adv_itr1.json	 pgd_5_adv_itr2.pt	  pgd_5_adv_itr5_model.pt
pgd_5_adv_itr1_model.pt  pgd_5_adv_itr3_model.pt  pgd_5_adv_itr5.pt
pgd_5_adv_itr1.pt	 pgd_5_adv_itr3.pt	  README.gdoc


In [ ]:
# Load
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd_5_adv_itr1.pt'
cifar_adv = torch.load(outfile_name)
len(cifar_adv)

782

In [ ]:
inputs, labels = cifar_adv[0]
labels

tensor([1, 5, 1, 2, 2, 7, 4, 1, 9, 1, 2, 6, 3, 9, 9, 0, 8, 8, 2, 4, 3, 9, 7, 4,
        6, 0, 4, 8, 0, 1, 4, 7, 0, 3, 5, 6, 1, 5, 8, 7, 1, 8, 9, 8, 8, 5, 1, 9,
        2, 0, 5, 1, 2, 1, 6, 2, 5, 2, 9, 4, 8, 5, 4, 4])

In [ ]:
t0 = time.time()

i = 0
correct = 0
total = 0
running_loss = 0.0
for (inputs, labels) in cifar_adv:
  # inputs & labels already on device
  # Adv training
  optimizer.zero_grad()
  inputs = inputs.to(device)
  labels = labels.to(device)

  # forward + backward + optimize
  outputs = ViT_model(inputs)
  loss = criterion(outputs, labels)
  loss.backward()
  optimizer.step()
  running_loss += loss.item()
  
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  # print("Here", np.sum(predicted == labels))
  if i % 150 == 0:
    print('Training accuracy of ViT on %d adv exs: %f %%' % (total, 100 * correct / total))
    print("running loss: ", running_loss)
  i += 1

print('Training accuracy on %d pgd-5 cifar-10 adv images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ATTA_5_ep1_train_acc"] = 100 * correct / total
t1 = time.time()
result_dict["ViT_ATTA_5_ep1_train_sec"] = t1 - t0
result_dict


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Training accuracy of ViT on 64 adv exs: 7.812500 %
running loss:  7.340195178985596
Training accuracy of ViT on 9664 adv exs: 10.885762 %
running loss:  357.84424209594727
Training accuracy of ViT on 19264 adv exs: 11.238580 %
running loss:  704.4925017356873
Training accuracy of ViT on 28864 adv exs: 11.089939 %
running loss:  1050.6746814250946
Training accuracy of ViT on 38464 adv exs: 11.020695 %
running loss:  1396.699630498886
Training accuracy of ViT on 48064 adv exs: 10.925017 %
running loss:  1742.6470937728882
Training accuracy on 50000 pgd-5 cifar-10 adv images: 10.960000 %


{'ViT_ATTA_5_ep1_train_acc': 10.96,
 'ViT_ATTA_5_ep1_train_sec': 1155.8513135910034}

In [ ]:
# Training V2 log
t0 = time.time()

i = 0
correct = 0
total = 0
running_loss = 0.0
for (inputs, labels) in cifar_adv:
  # inputs & labels already on device
  # Adv training
  optimizer.zero_grad()
  inputs = inputs.to(device)
  labels = labels.to(device)

  # forward + backward + optimize
  outputs = ViT_model(inputs)
  loss = criterion(outputs, labels)
  loss.backward()
  optimizer.step()
  running_loss += loss.item()
  
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  # print("Here", np.sum(predicted == labels))
  if i % 150 == 0:
    print('Training accuracy of ViT on %d adv exs: %f %%' % (total, 100 * correct / total))
    print("running loss: ", running_loss)
  i += 1

print('Training accuracy on %d pgd-5 cifar-10 adv images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ATTA_5_ep1_train_acc"] = 100 * correct / total
t1 = time.time()
result_dict["ViT_ATTA_5_ep1_train_sec"] = t1 - t0
result_dict


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Training accuracy of ViT on 64 adv exs: 15.625000 %
running loss:  7.470674991607666
Training accuracy of ViT on 9664 adv exs: 11.392798 %
running loss:  357.2774474620819
Training accuracy of ViT on 19264 adv exs: 11.290490 %
running loss:  703.5805416107178
Training accuracy of ViT on 28864 adv exs: 11.141907 %
running loss:  1049.879697561264
Training accuracy of ViT on 38464 adv exs: 10.947899 %
running loss:  1396.1210498809814
Training accuracy of ViT on 48064 adv exs: 10.956225 %
running loss:  1742.033133983612
Training accuracy on 50000 pgd-5 cifar-10 adv images: 10.938000 %


{'ATTA_ep5_Orig_Acc': 10.0,
 'Transformer': 98.74,
 'ViT_ATTA_5_ep1_train_acc': 10.938,
 'ViT_ATTA_5_ep1_train_sec': 656.953905582428}

In [ ]:
# Store model after itr1
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd_5_adv_itr1_v2_model.pt'
!touch $outfile_name
torch.save(ViT_model, outfile_name)
!ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

pgd_10_adv_test.pt	    pgd_5_adv_itr1_v2.pt     pgd_5_adv_itr4.pt
pgd_20_adv_test.pt	    pgd_5_adv_itr2_model.pt  pgd_5_adv_itr5_model.pt
pgd_5_adv_itr1.json	    pgd_5_adv_itr2.pt	     pgd_5_adv_itr5.pt
pgd_5_adv_itr1_model.pt     pgd_5_adv_itr3_model.pt  README.gdoc
pgd_5_adv_itr1.pt	    pgd_5_adv_itr3.pt
pgd_5_adv_itr1_v2_model.pt  pgd_5_adv_itr4_model.pt


In [ ]:
# Load model
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd_5_adv_itr1_model.pt'
ViT_model = torch.load(outfile_name)

# Evaluate ATTA Epoch 1 Model

In [ ]:
# Evaluate model on pgd-10 test adv
correct = 0
total = 0
for i, (inputs, labels) in enumerate(cifar_adv_test10):
  inputs, labels = inputs.to(device), labels.to(device)
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  if(i % 100 == 0):
    print('Accuracy on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

print('Accuracy ATTA5-ep1 ViT on %d cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ATTA_5_ep1_test_pgd10_acc"] = 100 * correct / total

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Accuracy on 64 cifar-10 test images: 4.687500 %
Accuracy on 6464 cifar-10 test images: 9.823639 %
Accuracy ATTA5-ep1 ViT on 10000 cifar-10 test images: 10.000000 %


In [ ]:
# Evaluate model on pgd-20 test adv
correct = 0
total = 0
for i, (inputs, labels) in enumerate(cifar_adv_test20):
  inputs, labels = inputs.to(device), labels.to(device)
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  if(i % 100 == 0):
    print('Accuracy on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

print('Accuracy ATTA5-ep1 ViT on %d cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ATTA_5_ep1_test_pgd20_acc"] = 100 * correct / total

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Accuracy on 64 cifar-10 test images: 4.687500 %
Accuracy on 6464 cifar-10 test images: 9.823639 %
Accuracy ATTA5-ep1 ViT on 10000 cifar-10 test images: 10.000000 %


# ATTA Training Epoch 2


In [ ]:
# ATTA generate attack ep2
i = 0
t0 = time.time()
for (inputs, labels) in cifar_adv:
  adv_images = atk_pgd_5(inputs, labels)
  cifar_adv[i] = (adv_images.cpu(), labels.cpu())
  if i % 100 == 0:
    print("Progress(%): ", 64*i/500)
  i += 1

time_gen_atk_2 = time.time() - t0
print("time lap(sec):", time_gen_atk_2)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Progress(%):  0.0
Progress(%):  16.0
Progress(%):  32.0
Progress(%):  48.0
Progress(%):  64.0
Progress(%):  80.0
Progress(%):  96.0
Progress(%):  112.0
time lap(sec): 8496.179186105728


In [ ]:
# Store
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd_5_adv_itr2.pt'
!touch $outfile_name
torch.save(cifar_adv, outfile_name)
!ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

pgd_10_adv_test.pt  pgd_5_adv_itr1.json      pgd_5_adv_itr1.pt	README.gdoc
pgd_20_adv_test.pt  pgd_5_adv_itr1_model.pt  pgd_5_adv_itr2.pt


In [ ]:
# Load
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd_5_adv_itr2.pt'
cifar_adv = torch.load(outfile_name)
len(cifar_adv)

782

In [ ]:
# ATTA train ep2
t0 = time.time()

i = 0
correct = 0
total = 0
running_loss = 0.0
for (inputs, labels) in cifar_adv:
  # inputs & labels already on device
  # Adv training
  optimizer.zero_grad()
  inputs = inputs.to(device)
  labels = labels.to(device)

  # forward + backward + optimize
  outputs = ViT_model(inputs)
  loss = criterion(outputs, labels)
  loss.backward()
  optimizer.step()
  running_loss += loss.item()
  
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  # print("Here", np.sum(predicted == labels))
  if i % 150 == 0:
    print('Training accuracy of ViT on %d adv exs: %f %%' % (total, 100 * correct / total))
    print("running loss: ", running_loss)
  i += 1

print('Training accuracy on %d pgd-5 cifar-10 adv images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ATTA_5_ep2_train_acc"] = 100 * correct / total
t1 = time.time()
result_dict["ViT_ATTA_5_ep2_train_sec"] = t1 - t0

# Store model after itr2
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd_5_adv_itr2_model.pt'
!touch $outfile_name
torch.save(ViT_model, outfile_name)
!ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

result_dict

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Training accuracy of ViT on 64 adv exs: 15.625000 %
running loss:  2.2928543090820312
Training accuracy of ViT on 9664 adv exs: 11.951573 %
running loss:  349.6787316799164
Training accuracy of ViT on 19264 adv exs: 12.998339 %
running loss:  691.1132817268372
Training accuracy of ViT on 28864 adv exs: 13.979351 %
running loss:  1028.301528453827
Training accuracy of ViT on 38464 adv exs: 13.753120 %
running loss:  1369.2223527431488
Training accuracy of ViT on 48064 adv exs: 15.446072 %
running loss:  1673.916489124298
Training accuracy on 50000 pgd-5 cifar-10 adv images: 15.862000 %
pgd_10_adv_test.pt   pgd_5_adv_itr1_model.pt  pgd_5_adv_itr2.pt
pgd_20_adv_test.pt   pgd_5_adv_itr1.pt	      README.gdoc
pgd_5_adv_itr1.json  pgd_5_adv_itr2_model.pt


{'ViT_ATTA_5_ep1_test_pgd20_acc': 10.0,
 'ViT_ATTA_5_ep2_train_acc': 15.862,
 'ViT_ATTA_5_ep2_train_sec': 2270.394117832184}

In [ ]:
# Load model
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd_5_adv_itr2_model.pt'
ViT_model = torch.load(outfile_name)

# Evaluate ATTA Epoch 2 Model


In [ ]:
# Evaluate model on pgd-10 test adv
correct = 0
total = 0
for i, (inputs, labels) in enumerate(cifar_adv_test10):
  inputs, labels = inputs.to(device), labels.to(device)
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  if(i % 100 == 0):
    print('Accuracy on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

print('Accuracy ATTA5-ep2 ViT on %d cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ATTA_5_ep2_test_pgd10_acc"] = 100 * correct / total

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Accuracy on 64 cifar-10 test images: 15.625000 %
Accuracy on 6464 cifar-10 test images: 13.474629 %
Accuracy ATTA5-ep2 ViT on 10000 cifar-10 test images: 13.450000 %


In [ ]:
# Evaluate model on pgd-20 test adv
correct = 0
total = 0
for i, (inputs, labels) in enumerate(cifar_adv_test20):
  inputs, labels = inputs.to(device), labels.to(device)
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  if(i % 100 == 0):
    print('Accuracy on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

print('Accuracy ATTA5-ep2 ViT on %d cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ATTA_5_ep2_test_pgd20_acc"] = 100 * correct / total

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Accuracy on 64 cifar-10 test images: 15.625000 %
Accuracy on 6464 cifar-10 test images: 13.180693 %
Accuracy ATTA5-ep2 ViT on 10000 cifar-10 test images: 13.120000 %


# ATTA Training Epoch 3 ~


In [ ]:
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
t0 = time.time()
for ep in range(3):
  print("##########    Epoch " + str(ep+3) + "    ##########")
  ts = time.time()
  i = 0
  for (inputs, labels) in cifar_adv:
    adv_images = atk_pgd_5(inputs, labels)
    cifar_adv[i] = (adv_images.cpu(), labels.cpu())
    if i % 100 == 0:
      print("Progress(%): ", 64*i/500)
    i += 1
  time_gen_atk_2 = time.time() - ts
  print("time lap(sec):", time_gen_atk_2)
  time_name = "ViT_ATTA_5_ep" + str(ep+3) + "_gen_atk_sec"
  result_dict[time_name] = time_gen_atk_2

  # Store cifar-10
  outfile_name = 'pgd_5_adv_itr' + str(ep+3) + '.pt'
  outfile_name = output_dir + outfile_name
  !touch $outfile_name
  torch.save(cifar_adv, outfile_name)
  !ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

  ts = time.time()
  i = 0
  correct = 0
  total = 0
  running_loss = 0.0
  for (inputs, labels) in cifar_adv:
    # inputs & labels already on device
    # Adv training
    optimizer.zero_grad()
    inputs = inputs.to(device)
    labels = labels.to(device)

    # forward + backward + optimize
    outputs = ViT_model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    
    with torch.no_grad():
      outputs = ViT_model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    # print("Here", np.sum(predicted == labels))
    if i % 150 == 0:
      print('Training accuracy of ViT on %d adv exs: %f %%' % (total, 100 * correct / total))
      print("running loss: ", running_loss)
    i += 1

  print('Training accuracy on %d pgd-5 cifar-10 adv images: %f %%' % (
      total,
      100 * correct / total))
  t1 = time.time()
  acc_name = "ViT_ATTA_5_ep" + str(ep+3) + "_train_acc"
  time_name = "ViT_ATTA_5_ep" + str(ep+3) + "_train_sec"
  result_dict[acc_name] = 100 * correct / total
  result_dict[time_name] = t1 - ts

  # Store model after itr ep+3
  outfile_name = 'pgd_5_adv_itr' + str(ep+3) + '_model.pt'
  outfile_name = output_dir + outfile_name
  !touch $outfile_name
  torch.save(ViT_model, outfile_name)
  !ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

t1 = time.time()
result_dict["ViT_3eps_ATTA_cifar10_sec"] = t1 - t0

##########    Epoch 3    ##########


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Progress(%):  0.0
Progress(%):  12.8
Progress(%):  25.6
Progress(%):  38.4
Progress(%):  51.2
Progress(%):  64.0
Progress(%):  76.8
Progress(%):  89.6
time lap(sec): 2409.723920583725
pgd_10_adv_test.pt   pgd_5_adv_itr1_model.pt  pgd_5_adv_itr2.pt
pgd_20_adv_test.pt   pgd_5_adv_itr1.pt	      pgd_5_adv_itr3.pt
pgd_5_adv_itr1.json  pgd_5_adv_itr2_model.pt  README.gdoc
Training accuracy of ViT on 64 adv exs: 17.187500 %
running loss:  3.2737040519714355
Training accuracy of ViT on 9664 adv exs: 19.474338 %
running loss:  324.05403101444244
Training accuracy of ViT on 19264 adv exs: 21.797135 %
running loss:  622.9230260848999
Training accuracy of ViT on 28864 adv exs: 23.330100 %
running loss:  911.7267687320709
Training accuracy of ViT on 38464 adv exs: 24.295445 %
running loss:  1198.0771329402924
Training accuracy of ViT on 48064 adv exs: 22.790446 %
running loss:  1530.6852853298187
Training accuracy on 50000 pgd-5 cifar-10 adv images: 22.488000 %
pgd_10_adv_test.pt	 pgd_5_adv_itr1.pt

# Evaluate ATTA Epoch 5 Model

In [ ]:
# Evaluate model on pgd-10 test adv
correct = 0
total = 0
for i, (inputs, labels) in enumerate(cifar_adv_test10):
  inputs, labels = inputs.to(device), labels.to(device)
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  if(i % 100 == 0):
    print('Accuracy on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

print('Accuracy ATTA5-ep5 ViT on %d cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ATTA_5_ep5_test_pgd10_acc"] = 100 * correct / total

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Accuracy on 64 cifar-10 test images: 14.062500 %
Accuracy on 6464 cifar-10 test images: 14.681312 %
Accuracy ATTA5-ep5 ViT on 10000 cifar-10 test images: 14.680000 %


In [ ]:
# Evaluate model on pgd-20 test adv
correct = 0
total = 0
for i, (inputs, labels) in enumerate(cifar_adv_test20):
  inputs, labels = inputs.to(device), labels.to(device)
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  if(i % 100 == 0):
    print('Accuracy on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

print('Accuracy ATTA5-ep5 ViT on %d cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ATTA_5_ep5_test_pgd20_acc"] = 100 * correct / total

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Accuracy on 64 cifar-10 test images: 12.500000 %
Accuracy on 6464 cifar-10 test images: 14.418317 %
Accuracy ATTA5-ep5 ViT on 10000 cifar-10 test images: 14.310000 %


# Evaluate Model on Original Data


In [ ]:
# Epoch 1 Model on Original test data
correct = 0
total = 0
with torch.no_grad():
  for i, (inputs, labels) in enumerate(cifar_testloader):
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
      outputs = ViT_model(inputs)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    if(i % 10 == 0):
      print('Accuracy of the network on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

print('Accuracy of the network on %d cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ATTA_ep5_Orig_Acc"] = 100 * correct / total

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Accuracy of the network on 64 cifar-10 test images: 4.687500 %
Accuracy of the network on 704 cifar-10 test images: 7.954545 %
Accuracy of the network on 1344 cifar-10 test images: 9.672619 %
Accuracy of the network on 1984 cifar-10 test images: 9.929435 %
Accuracy of the network on 2624 cifar-10 test images: 9.603659 %
Accuracy of the network on 3264 cifar-10 test images: 9.681373 %
Accuracy of the network on 3904 cifar-10 test images: 9.759221 %
Accuracy of the network on 4544 cifar-10 test images: 9.925176 %
Accuracy of the network on 5184 cifar-10 test images: 10.069444 %
Accuracy of the network on 5824 cifar-10 test images: 9.958791 %
Accuracy of the network on 6464 cifar-10 test images: 9.823639 %
Accuracy of the network on 7104 cifar-10 test images: 9.867680 %
Accuracy of the network on 7744 cifar-10 test images: 9.930269 %
Accuracy of the network on 8384 cifar-10 test images: 9.983302 %
Accuracy of the network on 9024 cifar-10 test images: 10.039894 %
Accuracy of the network on

In [ ]:
# Epoch 1 V2 Model on Original test data
correct = 0
total = 0
with torch.no_grad():
  for i, (inputs, labels) in enumerate(cifar_testloader):
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
      outputs = ViT_model(inputs)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    if(i % 10 == 0):
      print('Accuracy of the network on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

print('Accuracy of the network on %d cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ATTA_ep5_Orig_Acc"] = 100 * correct / total

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Accuracy of the network on 64 cifar-10 test images: 9.375000 %
Accuracy of the network on 704 cifar-10 test images: 10.369318 %
Accuracy of the network on 1344 cifar-10 test images: 10.416667 %
Accuracy of the network on 1984 cifar-10 test images: 9.879032 %
Accuracy of the network on 2624 cifar-10 test images: 9.984756 %
Accuracy of the network on 3264 cifar-10 test images: 9.957108 %
Accuracy of the network on 3904 cifar-10 test images: 9.810451 %
Accuracy of the network on 4544 cifar-10 test images: 9.925176 %
Accuracy of the network on 5184 cifar-10 test images: 9.780093 %
Accuracy of the network on 5824 cifar-10 test images: 9.872940 %
Accuracy of the network on 6464 cifar-10 test images: 9.962871 %
Accuracy of the network on 7104 cifar-10 test images: 10.050676 %
Accuracy of the network on 7744 cifar-10 test images: 10.020661 %
Accuracy of the network on 8384 cifar-10 test images: 9.947519 %
Accuracy of the network on 9024 cifar-10 test images: 9.929078 %
Accuracy of the network 

In [ ]:
# Epoch 2 Model on Original test data
correct = 0
total = 0
with torch.no_grad():
  for i, (inputs, labels) in enumerate(cifar_testloader):
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
      outputs = ViT_model(inputs)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    if(i % 10 == 0):
      print('Accuracy of the network on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

print('Accuracy of the network on %d cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ATTA_ep5_Orig_Acc"] = 100 * correct / total

In [ ]:
# Epoch 5 Model on Original test data
correct = 0
total = 0
with torch.no_grad():
  for i, (inputs, labels) in enumerate(cifar_testloader):
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
      outputs = ViT_model(inputs)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    if(i % 10 == 0):
      print('Accuracy of the network on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

print('Accuracy of the network on %d cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ATTA_ep5_Orig_Acc"] = 100 * correct / total

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Accuracy of the network on 64 cifar-10 test images: 12.500000 %
Accuracy of the network on 704 cifar-10 test images: 13.494318 %
Accuracy of the network on 1344 cifar-10 test images: 14.955357 %
Accuracy of the network on 1984 cifar-10 test images: 15.020161 %
Accuracy of the network on 2624 cifar-10 test images: 15.129573 %
Accuracy of the network on 3264 cifar-10 test images: 15.073529 %
Accuracy of the network on 3904 cifar-10 test images: 14.959016 %
Accuracy of the network on 4544 cifar-10 test images: 14.920775 %
Accuracy of the network on 5184 cifar-10 test images: 14.872685 %
Accuracy of the network on 5824 cifar-10 test images: 15.024038 %
Accuracy of the network on 6464 cifar-10 test images: 15.006188 %
Accuracy of the network on 7104 cifar-10 test images: 15.047860 %
Accuracy of the network on 7744 cifar-10 test images: 14.914773 %
Accuracy of the network on 8384 cifar-10 test images: 14.957061 %
Accuracy of the network on 9024 cifar-10 test images: 14.882535 %
Accuracy of t

In [ ]:
result_dict

{'Transformer': 14.91,
 'ViT_3eps_ATTA_cifar10_sec': 9184.282611370087,
 'ViT_ATTA_5_ep2_test_pgd10_acc': 13.45,
 'ViT_ATTA_5_ep2_test_pgd20_acc': 13.12,
 'ViT_ATTA_5_ep3_gen_atk_sec': 2409.723920583725,
 'ViT_ATTA_5_ep3_train_acc': 22.488,
 'ViT_ATTA_5_ep3_train_sec': 649.1576640605927,
 'ViT_ATTA_5_ep4_gen_atk_sec': 2411.9986884593964,
 'ViT_ATTA_5_ep4_train_acc': 15.032,
 'ViT_ATTA_5_ep4_train_sec': 648.385701417923,
 'ViT_ATTA_5_ep5_gen_atk_sec': 2400.8721063137054,
 'ViT_ATTA_5_ep5_test_pgd10_acc': 14.68,
 'ViT_ATTA_5_ep5_test_pgd20_acc': 14.31,
 'ViT_ATTA_5_ep5_train_acc': 17.956,
 'ViT_ATTA_5_ep5_train_sec': 650.7549364566803}